In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 493.7 kB/s eta 0:00:23
   - -------------------------------------- 0.5/11.5 MB 493.7 kB/s eta 0:00:23
   -- ------------------------------------- 0.8/11.5 MB 568.6 kB/s eta 0:00:19
   -- ------------------------------------- 0.8/11.5 MB 568.6 kB/s eta 0:00:19
   --- ------------------------------------ 1.0/11.5 MB 621.7 kB/s eta 0:00:17
   ---- ----------------------------------- 1.3/11.5 MB 671.3 kB/s eta 0:00:16
   ---- ----------------------------------- 1.3/11.5 MB 671.3 kB/s eta 0:00:16
   ----- ----------

In [3]:
import pandas as pd

In [ ]:
curr_data = pd.read_csv('../Notebooks/experiment/curr_data.csv')
curr_data_bb = pd.read_csv('../Notebooks/experiment/curr_data_bb.csv')
curr_data_rs = pd.read_csv('../Notebooks/experiment/curr_data_rs.csv')

In [11]:
curr_data_rs.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Revenue Share,Player ID.1,Stock,Sales,Leftovers,...,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,3,5,Retailer 1,90,90,0,...,82.20,24.80,305.4,411.0,0.573702,91.04,15.96,316.28,455.20,0.590035
1,1,2,Manufacturer 1,144,0,8,Retailer 1,150,144,6,...,100.50,49.50,402.0,354.0,0.468254,100.50,49.50,402.00,354.00,0.468254
2,1,3,Manufacturer 1,57,2,7,Retailer 1,0,0,0,...,0.00,110.00,0.0,0.0,NaN,92.30,17.70,241.50,536.10,0.689429
3,1,4,Manufacturer 1,149,2,6,Retailer 1,100,100,0,...,87.75,29.25,326.5,426.5,0.566401,94.89,22.11,335.34,452.34,0.574269
4,1,5,Manufacturer 1,135,3,4,Retailer 1,110,110,0,...,92.30,19.70,408.4,369.2,0.474794,93.09,18.91,408.72,372.36,0.476725


In [15]:


# Define categorization thresholds
def categorize_behavior(row):
    deviation = row["Under/Overstock_wrt_Optimal"]  # Deviation from optimal (expected sales)
    if row['Stock'] == 0:
        return 'Reject'
    elif deviation < -15:
        return 'Significant Underorder'
    elif -15 <= deviation <= 15:
        return 'Optimal'
    elif deviation > 15:
        return 'Overorder'
    else:
        return 'Unknown'

# Apply categorization to each dataset
curr_data['Behavioral Category'] = curr_data.apply(categorize_behavior, axis=1)
curr_data_bb['Behavioral Category'] = curr_data_bb.apply(categorize_behavior, axis=1)
curr_data_rs['Behavioral Category'] = curr_data_rs.apply(categorize_behavior, axis=1)

curr_data.to_csv('../Notebooks/experiment/curr_data.csv', index = False)
curr_data_bb.to_csv('../Notebooks/experiment/curr_data_bb.csv', index = False)
curr_data_rs.to_csv('../Notebooks/experiment/curr_data_rs.csv', index = False)


In [ ]:
behavioral_data = pd.read_csv('../adjusted_reponse_survey.csv')
historical_data_wholesale = pd.read_csv('../Notebooks/experiment/curr_data.csv')
historical_data_buyback = pd.read_csv('../Notebooks/experiment/curr_data_bb.csv')
historical_data_revenue_sharing = pd.read_csv('../Notebooks/experiment/curr_data_rs.csv')

In [ ]:
import numpy as np
import random
from collections import defaultdict

class QLearningAgent:
    def __init__(self, action_space, personality_traits=None, learning_rate=0.1, discount_factor=0.99,
                 epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01):
        self.action_space = action_space
        self.personality_traits = personality_traits or {}
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.q_table = defaultdict(float)
        self.estimated_human_traits = {'Risk Averse Coefficient': 0.5}
        self.human_actions = []
        self.behavior_change_threshold = 2  # Adjust as needed

    def _state_to_key(self, state):
        trait_values = tuple(self.personality_traits.values())
        return tuple(state) + trait_values

    def get_action(self, state):
        if self.detect_behavior_change():
            self.epsilon = min(1.0, self.epsilon * 1.1)  # Increase epsilon
        else:
            self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)  # Decay epsilon

        if np.random.rand() < self.epsilon:
            return self.action_space.sample()
        else:
            state_key = self._state_to_key(state)
            q_values = [self.q_table[(state_key, a)] for a in range(self.action_space.n)]
            return np.argmax(q_values)

    def update_q_table(self, state, action, reward, next_state):
        state_key = self._state_to_key(state)
        next_state_key = self._state_to_key(next_state)
        max_next_q = max([self.q_table[(next_state_key, a)] for a in range(self.action_space.n)])
        adjusted_reward = self.adjust_reward_based_on_traits(reward)
        self.q_table[(state_key, action)] += self.learning_rate * (
            adjusted_reward + self.discount_factor * max_next_q - self.q_table[(state_key, action)]
        )

    def update_estimated_human_traits(self, human_action):
        self.human_actions.append(human_action)
        if len(self.human_actions) > 10:
            recent_actions = self.human_actions[-10:]
            action_variance = np.var(recent_actions)
            max_variance = (self.action_space.n - 1) ** 2 / 12  # Variance of uniform distribution
            self.estimated_human_traits['Risk Averse Coefficient'] = max(0, min(1, 1 - action_variance / max_variance))

    def adjust_reward_based_on_traits(self, reward):
        risk_aversion = self.estimated_human_traits.get('Risk Averse Coefficient', 0.5)
        trait_factor = 1.0 + (0.5 - risk_aversion)
        adjusted_reward = reward * trait_factor
        return adjusted_reward

    def detect_behavior_change(self):
        if len(self.human_actions) < 20:
            return False
        recent_actions = self.human_actions[-10:]
        previous_actions = self.human_actions[-20:-10]
        recent_mean = np.mean(recent_actions)
        previous_mean = np.mean(previous_actions)
        if abs(recent_mean - previous_mean) > self.behavior_change_threshold:
            return True
        return False

In [131]:
def extract_traits(behavioral_data):
    """
    Extract traits for all manufacturers and retailers from the behavioral data.
    Returns dictionaries of traits for manufacturers and retailers indexed by PLAYER NAME.
    """
    manufacturers = behavioral_data[[
        'Manufacturer_Self Esteem Average',
        'Manufacturer_Regret Scale Average',
        'Manufacturer_Risk Averse Coefficient',
        'Manufacturer_Fairness Index'
    ]].rename(lambda col: col.replace('Manufacturer_', ''), axis=1)

    retailers = behavioral_data[[
        'Retailer_Self Esteem Average',
        'Retailer_Regret Scale Average',
        'Retailer_Risk Averse Coefficient',
        'Retailer_Fairness Index'
    ]].rename(lambda col: col.replace('Retailer_', ''), axis=1)

    # Return as dictionaries indexed by PLAYER NAME
    manufacturer_traits = manufacturers.to_dict(orient='index')
    retailer_traits = retailers.to_dict(orient='index')

    return manufacturer_traits, retailer_traits

In [132]:
"""

# Extract traits
manufacturer_traits, retailer_traits = extract_traits(behavioral_data)

# Sample traits for inspection
sample_manufacturer = list(manufacturer_traits.keys())[0]
sample_retailer = list(retailer_traits.keys())[0]

sample_manufacturer_traits = manufacturer_traits[sample_manufacturer]
sample_retailer_traits = retailer_traits[sample_retailer]

"""

'\n\n# Extract traits\nmanufacturer_traits, retailer_traits = extract_traits(behavioral_data)\n\n# Sample traits for inspection\nsample_manufacturer = list(manufacturer_traits.keys())[0]\nsample_retailer = list(retailer_traits.keys())[0]\n\nsample_manufacturer_traits = manufacturer_traits[sample_manufacturer]\nsample_retailer_traits = retailer_traits[sample_retailer]\n\n'

In [133]:
# Redefine the environment class and test setup
import numpy as np
from gym import Env, spaces

class SupplyChainEnv(Env):
    """
    Custom Environment for the supply chain game.
    """
    def __init__(self, contract_type="wholesale"):
        super(SupplyChainEnv, self).__init__()
        self.contract_type = contract_type
        self.max_stock = 150
        self.max_price = 12
        self.max_rounds = 40
        self.current_round = 0

        if self.contract_type == "wholesale":
            self.manufacturer_action_space = spaces.Discrete(self.max_price + 1)
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "buyback":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "revenue-sharing":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        else:
            raise ValueError("Invalid contract type.")

        self.observation_space = spaces.Box(low=0, high=150, shape=(5,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.demand = np.random.randint(50, 151)
        self.state = np.array([self.demand, 0, 0, 0, 0])
        self.current_round = 0
        return self.state

    def step(self, actions):
        manufacturer_action, retailer_action = actions
        Q = retailer_action
        sales = min(Q, self.demand)
        leftovers = Q - sales
        c = 5
        p = self.max_price

        if self.contract_type == "wholesale":
            # Extract wholesale price
            w = manufacturer_action
            b = 0
            r = 0

            # Retailer payoff
            retailer_profit = p * sales - w * Q

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q

        elif self.contract_type == "buyback":
            # Extract wholesale price and buyback price
            w, b = manufacturer_action
            r = 0

            # Enforce constraint: buyback price must not exceed wholesale price
            if b > w:
                b = w  # Adjust buyback price to wholesale price

            # Retailer payoff
            retailer_profit = p * sales - w * Q + b * leftovers

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q - b * leftovers

        elif self.contract_type == "revenue-sharing":
            # Extract wholesale price and revenue share
            w, r = manufacturer_action
            b = 0

            # Enforce constraint: revenue share must not exceed (retail price - wholesale price)
            max_revenue_share = p - w
            if r > max_revenue_share:
                r = max_revenue_share

            # Retailer payoff
            retailer_profit = p * sales - w * Q - r * sales

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q + r * sales

        self.state = np.array([self.demand, w, b, r, retailer_profit, manufacturer_profit])
        self.current_round += 1
        self.demand = np.random.randint(50, 151)
        done = self.current_round >= self.max_rounds

        return self.state, (manufacturer_profit, retailer_profit), done, {}

"""
# Initialize the environment
env = SupplyChainEnv(contract_type="wholesale")

# Initialize agents with personality traits
manufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits=sample_manufacturer_traits)
retailer_agent = QLearningAgent(env.retailer_action_space, personality_traits=sample_retailer_traits)
"""



'\n# Initialize the environment\nenv = SupplyChainEnv(contract_type="wholesale")\n\n# Initialize agents with personality traits\nmanufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits=sample_manufacturer_traits)\nretailer_agent = QLearningAgent(env.retailer_action_space, personality_traits=sample_retailer_traits)\n'

In [134]:
behavioral_data.columns = behavioral_data.columns.str.strip()
historical_data_wholesale.columns = historical_data_wholesale.columns.str.strip()
historical_data_buyback.columns = historical_data_buyback.columns.str.strip()
historical_data_revenue_sharing.columns = historical_data_revenue_sharing.columns.str.strip()

In [135]:
historical_data_wholesale.columns = historical_data_wholesale.columns.str.replace(' ', '_')
historical_data_buyback.columns = historical_data_buyback.columns.str.replace(' ', '_')
historical_data_revenue_sharing.columns = historical_data_revenue_sharing.columns.str.replace(' ', '_')

In [136]:

# Extract behavioral data
behavioral_data = behavioral_data[['PLAYER NAME', 
                                   'Manufacturer_Self Esteem Average', 
                                   'Manufacturer_Regret Scale Average', 
                                   'Manufacturer_Risk Averse Coefficient', 
                                   'Manufacturer_Fairness Index', 
                                   'Retailer_Self Esteem Average', 
                                   'Retailer_Regret Scale Average', 
                                   'Retailer_Risk Averse Coefficient', 
                                   'Retailer_Fairness Index']].set_index('PLAYER NAME')

# Function to extract historical data for Q-learning
def preprocess_historical_data(df, contract_type):
    data = []
    for _, row in df.iterrows():
        # State variables
        state = [
            row['Demand'], 
            row['Wholesale_p.'], 
            row.get('Buyback_p.', 0),  # Default to 0 if not present
            row.get('Revenue_Share', 0)  # Default to 0 if not present
        ]
        
        # Actions
        manufacturer_action = (
            row['Wholesale_p.'], 
            row.get('Buyback_p.', row.get('Revenue_Share', 0))
        )
        retailer_action = row['Stock']
        
        # Rewards
        manufacturer_reward = row['Realized_Mfg_Profit']
        retailer_reward = row['Realized_Retailer_Profit']
        
        # Next state (mocked as same for simplicity, typically inferred)
        next_state = state
        
        data.append((state, manufacturer_action, retailer_action, (manufacturer_reward, retailer_reward), next_state))
    return data

# Process historical data for each contract type
historical_data_wholesale = preprocess_historical_data(historical_data_wholesale, "wholesale")
historical_data_buyback = preprocess_historical_data(historical_data_buyback, "buyback")
historical_data_revenue_sharing = preprocess_historical_data(historical_data_revenue_sharing, "revenue-sharing")

# Combine historical data
historical_data = historical_data_wholesale + historical_data_buyback + historical_data_revenue_sharing


# Initialize agents with personality traits
manufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits={'risk_aversion': 0.7, 'fairness': 0.5})
retailer_agent = QLearningAgent(env.retailer_action_space, personality_traits={'risk_aversion': 0.3, 'self_esteem': 0.8})

# Pre-train Q-table using historical data
for record in historical_data:
    state, manufacturer_action, retailer_action, rewards, next_state = record
    manufacturer_reward, retailer_reward = rewards

    # Update Q-table for manufacturer
    manufacturer_agent.q_table[(manufacturer_agent._state_to_key(state), manufacturer_action)] = manufacturer_reward

    # Update Q-table for retailer
    retailer_agent.q_table[(retailer_agent._state_to_key(state), retailer_action)] = retailer_reward

In [139]:
# Extract traits
manufacturer_traits, retailer_traits = extract_traits(behavioral_data)

# Sample traits for inspection
sample_manufacturer = list(manufacturer_traits.keys())[0]
sample_retailer = list(retailer_traits.keys())[0]

sample_manufacturer_traits = manufacturer_traits[sample_manufacturer]
sample_retailer_traits = retailer_traits[sample_retailer]

# Initialize the environment
env = SupplyChainEnv(contract_type="wholesale")

# Initialize agents with personality traits
manufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits=sample_manufacturer_traits)
retailer_agent = QLearningAgent(env.retailer_action_space, personality_traits=sample_retailer_traits)


In [ ]:
# bu errorlu olmayan, ama yanlış
# Import necessary libraries
import numpy as np
import pandas as pd
from collections import defaultdict
from gym import Env, spaces
import random

# Define the SupplyChainEnv class
class SupplyChainEnv(Env):
    """
    Custom Environment for the supply chain game.
    """
    def __init__(self, contract_type="wholesale"):
        super(SupplyChainEnv, self).__init__()
        self.contract_type = contract_type
        self.max_stock = 150
        self.max_price = 12
        self.max_rounds = 40
        self.current_round = 0

        if self.contract_type == "wholesale":
            self.manufacturer_action_space = spaces.Discrete(self.max_price + 1)
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "buyback":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "revenue-sharing":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        else:
            raise ValueError("Invalid contract type.")

        self.observation_space = spaces.Box(low=0, high=150, shape=(5,), dtype=np.float32)
        self.reset()

    def reset(self):
        self.demand = np.random.randint(50, 151)
        self.state = np.array([self.demand, 0, 0, 0, 0])  # Initial state
        self.current_round = 0
        return self.state

    def step(self, actions):
        manufacturer_action, retailer_action = actions
        Q = retailer_action
        sales = min(Q, self.demand)
        leftovers = Q - sales
        c = 5  # Manufacturer's cost
        p = 12  # Retail price

        if self.contract_type == "wholesale":
            # Extract wholesale price
            w = manufacturer_action
            b = 0
            r = 0

            # Retailer payoff
            retailer_profit = p * sales - w * Q

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q

        elif self.contract_type == "buyback":
            # Extract wholesale price and buyback price
            w, b = manufacturer_action
            r = 0

            # Enforce constraint: buyback price must not exceed wholesale price
            if b > w:
                b = w  # Adjust buyback price to wholesale price

            # Retailer payoff
            retailer_profit = p * sales - w * Q + b * leftovers

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q - b * leftovers

        elif self.contract_type == "revenue-sharing":
            # Extract wholesale price and revenue share
            w, r = manufacturer_action
            b = 0

            # Enforce constraint: revenue share must not exceed (retail price - wholesale price)
            max_revenue_share = p - w
            if r > max_revenue_share:
                r = max_revenue_share

            # Retailer payoff
            retailer_profit = p * sales - w * Q - r * sales

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q + r * sales

        # Update the state
        self.state = np.array([self.demand, w, b, r, retailer_profit, manufacturer_profit])
        self.current_round += 1
        self.demand = np.random.randint(50, 151)  # Generate new demand for next round
        done = self.current_round >= self.max_rounds

        # For simplicity, we return the same state as next_state
        next_state = self.state.copy()

        return next_state, (manufacturer_profit, retailer_profit), done, {}

# Define the QLearningAgent class with Trait-Driven Reward Modulation and dynamic Exploration vs. Exploitation Balance
class QLearningAgent:
    def __init__(self, action_space, personality_traits=None, learning_rate=0.1, discount_factor=0.99,
                 epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01):
        self.action_space = action_space
        self.personality_traits = personality_traits or {}
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.initial_epsilon = epsilon  # Store initial epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.q_table = defaultdict(float)
        self.estimated_human_traits = {'Risk Averse Coefficient': 0.5}
        self.human_actions = []
        self.behavior_change_threshold = 2  # Adjust as needed

    def _state_to_key(self, state):
        trait_values = tuple(self.personality_traits.values())
        return tuple(state) + trait_values

    def get_action(self, state):
        if self.detect_behavior_change():
            self.epsilon = min(1.0, self.epsilon * 1.1)  # Increase epsilon to promote exploration
        else:
            self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)  # Decay epsilon

        if np.random.rand() < self.epsilon:
            # Explore: choose a random action
            if isinstance(self.action_space, spaces.Discrete):
                return self.action_space.sample()
            elif isinstance(self.action_space, spaces.MultiDiscrete):
                return tuple(self.action_space.sample())
        else:
            # Exploit: choose the action with the highest Q-value
            state_key = self._state_to_key(state)
            possible_actions = self.get_possible_actions()
            q_values = [self.q_table[(state_key, a)] for a in possible_actions]
            max_q = max(q_values)
            max_actions = [a for a, q in zip(possible_actions, q_values) if q == max_q]
            chosen_action = random.choice(max_actions)
            return chosen_action

    def update_q_table(self, state, action, reward, next_state):
        state_key = self._state_to_key(state)
        next_state_key = self._state_to_key(next_state)

        # Get possible actions for next state
        possible_actions = self.get_possible_actions()
        max_next_q = max([self.q_table[(next_state_key, a)] for a in possible_actions])

        # Adjust the reward based on estimated human traits
        adjusted_reward = self.adjust_reward_based_on_traits(reward)

        # Update Q-value using the adjusted reward
        self.q_table[(state_key, action)] += self.learning_rate * (
            adjusted_reward + self.discount_factor * max_next_q - self.q_table[(state_key, action)]
        )

    def update_estimated_human_traits(self, human_action):
        self.human_actions.append(human_action)
        if len(self.human_actions) > 10:
            recent_actions = self.human_actions[-10:]
            action_variance = np.var(recent_actions)
            max_variance = (self.action_space.n - 1) ** 2 / 12  # Variance of uniform distribution
            self.estimated_human_traits['Risk Averse Coefficient'] = max(0, min(1, 1 - action_variance / max_variance))

    def adjust_reward_based_on_traits(self, reward):
        risk_aversion = self.estimated_human_traits.get('Risk Averse Coefficient', 0.5)
        trait_factor = 1.0 + (0.5 - risk_aversion)  # Adjust factor between 0.5 and 1.5
        adjusted_reward = reward * trait_factor
        return adjusted_reward

    def detect_behavior_change(self):
        if len(self.human_actions) < 20:
            return False
        recent_actions = self.human_actions[-10:]
        previous_actions = self.human_actions[-20:-10]
        recent_mean = np.mean(recent_actions)
        previous_mean = np.mean(previous_actions)
        if abs(recent_mean - previous_mean) > self.behavior_change_threshold:
            return True
        return False

    def get_possible_actions(self):
        if isinstance(self.action_space, spaces.Discrete):
            return range(self.action_space.n)
        elif isinstance(self.action_space, spaces.MultiDiscrete):
            ranges = [range(n) for n in self.action_space.nvec]
            possible_actions = list(itertools.product(*ranges))
            return possible_actions

# Import itertools for possible actions
import itertools

# Function to extract traits for all manufacturers and retailers from the behavioral data
def extract_traits(behavioral_data):
    # Extract manufacturer traits
    manufacturer_traits = behavioral_data[[
        'Manufacturer_Self Esteem Average',
        'Manufacturer_Regret Scale Average',
        'Manufacturer_Risk Averse Coefficient',
        'Manufacturer_Fairness Index'
    ]].rename(columns=lambda col: col.replace('Manufacturer_', '')).to_dict(orient='index')

    # Extract retailer traits
    retailer_traits = behavioral_data[[
        'Retailer_Self Esteem Average',
        'Retailer_Regret Scale Average',
        'Retailer_Risk Averse Coefficient',
        'Retailer_Fairness Index'
    ]].rename(columns=lambda col: col.replace('Retailer_', '')).to_dict(orient='index')

    return manufacturer_traits, retailer_traits

# Load behavioral data
behavioral_data = pd.read_csv('../adjusted_reponse_survey.csv')
behavioral_data.columns = behavioral_data.columns.str.strip()

# Extract traits
manufacturer_traits_dict, retailer_traits_dict = extract_traits(behavioral_data.set_index('PLAYER NAME'))

# For simplicity, select a sample manufacturer and retailer
sample_manufacturer = list(manufacturer_traits_dict.keys())[0]
sample_retailer = list(retailer_traits_dict.keys())[0]

sample_manufacturer_traits = manufacturer_traits_dict[sample_manufacturer]
sample_retailer_traits = retailer_traits_dict[sample_retailer]

# Load and preprocess historical data
def preprocess_historical_data(df, contract_type):
    data = []
    for _, row in df.iterrows():
        # State variables
        state = [
            row['Demand'],
            row['Wholesale_p.'],
            row.get('Buyback_p.', 0),  # Default to 0 if not present
            row.get('Revenue_Share', 0)  # Default to 0 if not present
        ]

        # Actions
        if contract_type == "wholesale":
            manufacturer_action = row['Wholesale_p.']
        else:
            manufacturer_action = (row['Wholesale_p.'], row.get('Buyback_p.', row.get('Revenue_Share', 0)))

        retailer_action = row['Stock']

        # Rewards
        manufacturer_reward = row['Realized_Mfg_Profit']
        retailer_reward = row['Realized_Retailer_Profit']

        # Next state (for simplicity, we use the same state)
        next_state = state

        data.append((state, manufacturer_action, retailer_action, (manufacturer_reward, retailer_reward), next_state))
    return data

# Load historical data
historical_data_wholesale = pd.read_csv('../Notebooks/experiment/curr_data.csv')
historical_data_buyback = pd.read_csv('../Notebooks/experiment/curr_data_bb.csv')
historical_data_revenue_sharing = pd.read_csv('../Notebooks/experiment/curr_data_rs.csv')

# Clean column names
historical_data_wholesale.columns = historical_data_wholesale.columns.str.strip().str.replace(' ', '_')
historical_data_buyback.columns = historical_data_buyback.columns.str.strip().str.replace(' ', '_')
historical_data_revenue_sharing.columns = historical_data_revenue_sharing.columns.str.strip().str.replace(' ', '_')

# Process historical data
historical_data_wholesale = preprocess_historical_data(historical_data_wholesale, "wholesale")
historical_data_buyback = preprocess_historical_data(historical_data_buyback, "buyback")
historical_data_revenue_sharing = preprocess_historical_data(historical_data_revenue_sharing, "revenue-sharing")

# Combine historical data
historical_data = historical_data_wholesale + historical_data_buyback + historical_data_revenue_sharing

# Initialize the environment
env = SupplyChainEnv(contract_type="wholesale")  # You can change the contract type as needed

# Initialize agents with personality traits
manufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits=sample_manufacturer_traits)
retailer_agent = QLearningAgent(env.retailer_action_space, personality_traits=sample_retailer_traits)

# Pre-train the agents using historical data
for record in historical_data:
    state, manufacturer_action, retailer_action, rewards, next_state = record
    manufacturer_reward, retailer_reward = rewards

    # Update Q-table for manufacturer
    state_key = manufacturer_agent._state_to_key(state)
    manufacturer_agent.q_table[(state_key, manufacturer_action)] = manufacturer_reward

    # Update Q-table for retailer
    state_key = retailer_agent._state_to_key(state)
    retailer_agent.q_table[(state_key, retailer_action)] = retailer_reward

# Main training loop
num_episodes = 100000  # Number of episodes for training
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        # Agents select actions
        manufacturer_action = manufacturer_agent.get_action(state)
        retailer_action = retailer_agent.get_action(state)

        # Environment step
        next_state, rewards, done, info = env.step((manufacturer_action, retailer_action))
        manufacturer_reward, retailer_reward = rewards

        # Update estimated human traits (assuming retailer is human for this example)
        # In this example, both agents are AI, so we can simulate human actions
        # For real human interaction, you would get human_action from user input
        manufacturer_agent.update_estimated_human_traits(retailer_action)
        retailer_agent.update_estimated_human_traits(manufacturer_action)

        # Update Q-tables
        manufacturer_agent.update_q_table(state, manufacturer_action, manufacturer_reward, next_state)
        retailer_agent.update_q_table(state, retailer_action, retailer_reward, next_state)

        # Move to the next state
        state = next_state

    # Optionally, print progress
    print(f"Episode {episode + 1}/{num_episodes} completed.")

# The agents are now trained and can be used to play the game against a human
# For example, to have the AI manufacturer play against a human retailer:

# Initialize environment
env = SupplyChainEnv(contract_type="wholesale")  # Adjust contract type as needed

# Reset environment
state = env.reset()
done = False

print("Game start. You are the Retailer.")

while not done:
    # Manufacturer (AI) selects action
    manufacturer_action = manufacturer_agent.get_action(state)

    # Human (Retailer) input
    print(f"Demand: {state[0]}, Manufacturer offers wholesale price: {manufacturer_action}")
    retailer_action = int(input("Enter your order quantity (0-150): "))

    # Ensure the action is within the valid range
    retailer_action = max(0, min(env.retailer_action_space.n - 1, retailer_action))

    # Environment step
    next_state, rewards, done, info = env.step((manufacturer_action, retailer_action))
    manufacturer_reward, retailer_reward = rewards

    # Update estimated human traits
    manufacturer_agent.update_estimated_human_traits(retailer_action)

    # Update Q-table for manufacturer agent
    manufacturer_agent.update_q_table(state, manufacturer_action, manufacturer_reward, next_state)

    # Display results
    print(f"Retailer profit: {retailer_reward}, Manufacturer profit: {manufacturer_reward}")

    # Move to the next state
    state = next_state

print("Game over.")



Wholesale Data Columns: Index(['Experiment', 'Period', 'Player_ID', 'Demand', 'Wholesale_p.',
       'Player_ID.1', 'Stock', 'Sales', 'Leftovers', 'Unmet_Demand',
       'Realized_Mfg_Profit', 'Realized_Retailer_Profit',
       'Realized_Mfg_Profit_Share', 'Optimal_Stock',
       'Under/Overstock_wrt_Optimal', 'Expected_Sales', 'Expected_Leftovers',
       'Expected_Retailer_Profit', 'Expected_Mfg_Profit',
       'Expected_Mfg._Profit_Share', 'Predicted_Sales', 'Predicted_Leftovers',
       'Predicted_Retailer_Profit', 'Predicted_Mfg_Profit',
       'Realized_Retailer_Profit', 'Predicted_Mfg._Profit_Share',
       'Behavioral_Category'],
      dtype='object')
Buyback Data Columns: Index(['Experiment', 'Period', 'Player_ID', 'Demand', 'Wholesale_p.',
       'Buyback_p.', 'Player_ID.1', 'Stock', 'Sales', 'Leftovers',
       'Unmet_Demand', 'Realized_Mfg_Profit', 'Realized_Retailer_Profit',
       'Realized_Retailer_Profit', 'Realized_Mfg_Profit_Share',
       'Optimal_Stock', 'Under/Over

TypeError: cannot convert the series to <class 'float'>

In [ ]:
# bu errorlu olan, ama doğru
# Import necessary libraries
import numpy as np
import pandas as pd
from collections import defaultdict
from gym import Env, spaces
import random

# Define the SupplyChainEnv class
class SupplyChainEnv(Env):
    """
    Custom Environment for the supply chain game.
    """
    def __init__(self, contract_type="wholesale"):
        super(SupplyChainEnv, self).__init__()
        self.contract_type = contract_type
        self.max_stock = 150
        self.max_price = 12
        self.max_rounds = 40
        self.current_round = 0

        if self.contract_type == "wholesale":
            self.manufacturer_action_space = spaces.Discrete(self.max_price + 1)
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "buyback":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        elif self.contract_type == "revenue-sharing":
            self.manufacturer_action_space = spaces.MultiDiscrete([self.max_price + 1, self.max_price + 1])
            self.retailer_action_space = spaces.Discrete(self.max_stock + 1)
        else:
            raise ValueError("Invalid contract type.")

        # The state does not include the demand since it's unknown before decisions are made
        self.observation_space = spaces.Box(low=0, high=150, shape=(4,), dtype=np.float32)
        self.reset()

    def reset(self):
        # At reset, we do not generate demand or include it in the state
        self.state = np.array([0, 0, 0, 0])  # State: [Wholesale Price, Buyback Price, Revenue Share, Previous Profit]
        self.current_round = 0
        return self.state

    def step(self, actions):
        manufacturer_action, retailer_action = actions

        # Unpack actions
        if self.contract_type == "wholesale":
            w = manufacturer_action  # Wholesale price
            b = 0
            r = 0
        elif self.contract_type == "buyback":
            w, b = manufacturer_action  # Wholesale price and Buyback price
            r = 0
        elif self.contract_type == "revenue-sharing":
            w, r = manufacturer_action  # Wholesale price and Revenue share
            b = 0

        Q = retailer_action  # Retailer's order quantity

        # Generate demand AFTER decisions are made
        self.demand = np.random.randint(50, 151)

        # Calculate sales and leftovers
        sales = min(Q, self.demand)
        leftovers = Q - sales
        c = 3  # Manufacturer's production cost
        p = 12  # Retail price

        if self.contract_type == "wholesale":
            # Retailer payoff
            retailer_profit = p * sales - w * Q

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q

        elif self.contract_type == "buyback":
            # Enforce constraint: buyback price must not exceed wholesale price
            if b > w:
                b = w  # Adjust buyback price to wholesale price

            # Retailer payoff
            retailer_profit = p * sales - w * Q + b * leftovers

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q - b * leftovers

        elif self.contract_type == "revenue-sharing":
            # Enforce constraint: revenue share must not exceed (retail price - wholesale price)
            max_revenue_share = p - w
            if r > max_revenue_share:
                r = max_revenue_share

            # Retailer payoff
            retailer_profit = (p - r) * sales - w * Q

            # Manufacturer payoff
            manufacturer_profit = (w - c) * Q + r * sales

        # Update the state
        # State now includes previous profits as feedback
        self.state = np.array([w, b, r, retailer_profit])

        self.current_round += 1
        done = self.current_round >= self.max_rounds

        # For simplicity, next state is the current state (excluding demand)
        next_state = self.state.copy()

        # Return next_state, rewards, done, info
        return next_state, (manufacturer_profit, retailer_profit), done, {}

# The rest of the code remains largely the same, but we need to adjust how states are represented
# Update the QLearningAgent class if necessary

# Import itertools for possible actions
import itertools

# Define the QLearningAgent class with Trait-Driven Reward Modulation and dynamic Exploration vs. Exploitation Balance
class QLearningAgent:
    def __init__(self, action_space, personality_traits=None, learning_rate=0.1, discount_factor=0.99,
                 epsilon=1.0, epsilon_decay=0.995, min_epsilon=0.01):
        self.action_space = action_space
        self.personality_traits = personality_traits or {}
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.initial_epsilon = epsilon  # Store initial epsilon
        self.epsilon_decay = epsilon_decay
        self.min_epsilon = min_epsilon
        self.q_table = defaultdict(float)
        self.estimated_human_traits = {'Risk Averse Coefficient': 0.5}
        self.human_actions = []
        self.behavior_change_threshold = 2  # Adjust as needed

    def _state_to_key(self, state):
        trait_values = tuple(self.personality_traits.values())
        return tuple(state) + trait_values

    def get_action(self, state):
        if self.detect_behavior_change():
            self.epsilon = min(1.0, self.epsilon * 1.1)  # Increase epsilon to promote exploration
        else:
            self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)  # Decay epsilon

        if np.random.rand() < self.epsilon:
            # Explore: choose a random action
            if isinstance(self.action_space, spaces.Discrete):
                return self.action_space.sample()
            elif isinstance(self.action_space, spaces.MultiDiscrete):
                return tuple(self.action_space.sample())
        else:
            # Exploit: choose the action with the highest Q-value
            state_key = self._state_to_key(state)
            possible_actions = self.get_possible_actions()
            q_values = [self.q_table[(state_key, a)] for a in possible_actions]
            max_q = max(q_values)
            max_actions = [a for a, q in zip(possible_actions, q_values) if q == max_q]
            chosen_action = random.choice(max_actions)
            return chosen_action

    def update_q_table(self, state, action, reward, next_state):
        state_key = self._state_to_key(state)
        next_state_key = self._state_to_key(next_state)

        # Get possible actions for next state
        possible_actions = self.get_possible_actions()
        max_next_q = max([self.q_table[(next_state_key, a)] for a in possible_actions])

        # Adjust the reward based on estimated human traits
        adjusted_reward = self.adjust_reward_based_on_traits(reward)

        # Update Q-value using the adjusted reward
        self.q_table[(state_key, action)] += self.learning_rate * (
            adjusted_reward + self.discount_factor * max_next_q - self.q_table[(state_key, action)]
        )

    def update_estimated_human_traits(self, human_action):
        self.human_actions.append(human_action)
        if len(self.human_actions) > 10:
            recent_actions = self.human_actions[-10:]
            action_variance = np.var(recent_actions)
            max_variance = (self.action_space.n - 1) ** 2 / 12  # Variance of uniform distribution
            self.estimated_human_traits['Risk Averse Coefficient'] = max(0, min(1, 1 - action_variance / max_variance))

    def adjust_reward_based_on_traits(self, reward):
        risk_aversion = self.estimated_human_traits.get('Risk Averse Coefficient', 0.5)
        trait_factor = 1.0 + (0.5 - risk_aversion)  # Adjust factor between 0.5 and 1.5
        adjusted_reward = reward * trait_factor
        return adjusted_reward

    def detect_behavior_change(self):
        if len(self.human_actions) < 20:
            return False
        recent_actions = self.human_actions[-10:]
        previous_actions = self.human_actions[-20:-10]
        recent_mean = np.mean(recent_actions)
        previous_mean = np.mean(previous_actions)
        if abs(recent_mean - previous_mean) > self.behavior_change_threshold:
            return True
        return False

    def get_possible_actions(self):
        if isinstance(self.action_space, spaces.Discrete):
            return range(self.action_space.n)
        elif isinstance(self.action_space, spaces.MultiDiscrete):
            ranges = [range(n) for n in self.action_space.nvec]
            possible_actions = list(itertools.product(*ranges))
            return possible_actions

# Function to extract traits for all manufacturers and retailers from the behavioral data
def extract_traits(behavioral_data):
    # Extract manufacturer traits
    manufacturer_traits = behavioral_data[[
        'Manufacturer_Self Esteem Average',
        'Manufacturer_Regret Scale Average',
        'Manufacturer_Risk Averse Coefficient',
        'Manufacturer_Fairness Index'
    ]].rename(columns=lambda col: col.replace('Manufacturer_', '')).to_dict(orient='index')

    # Extract retailer traits
    retailer_traits = behavioral_data[[
        'Retailer_Self Esteem Average',
        'Retailer_Regret Scale Average',
        'Retailer_Risk Averse Coefficient',
        'Retailer_Fairness Index'
    ]].rename(columns=lambda col: col.replace('Retailer_', '')).to_dict(orient='index')

    return manufacturer_traits, retailer_traits

# Load behavioral data
behavioral_data = pd.read_csv('../adjusted_reponse_survey.csv')
behavioral_data.columns = behavioral_data.columns.str.strip()

# Extract traits
manufacturer_traits_dict, retailer_traits_dict = extract_traits(behavioral_data.set_index('PLAYER NAME'))

# For simplicity, select a sample manufacturer and retailer
sample_manufacturer = list(manufacturer_traits_dict.keys())[0]
sample_retailer = list(retailer_traits_dict.keys())[0]

sample_manufacturer_traits = manufacturer_traits_dict[sample_manufacturer]
sample_retailer_traits = retailer_traits_dict[sample_retailer]

# Load and preprocess historical data
def preprocess_historical_data(df, contract_type):
    data = []
    for _, row in df.iterrows():
        # Ensure that we get scalar values by converting to float
        wholesale_price = float(row['Wholesale_p.'])
        buyback_price = float(row['Buyback_p.']) if 'Buyback_p.' in row and not pd.isnull(row['Buyback_p.']) else 0.0
        revenue_share = float(row['Revenue_Share']) if 'Revenue_Share' in row and not pd.isnull(row['Revenue_Share']) else 0.0
        previous_retailer_profit = float(row['Realized_Retailer_Profit'])
        stock = int(row['Stock'])
        realized_mfg_profit = float(row['Realized_Mfg_Profit'])
        realized_retailer_profit = float(row['Realized_Retailer_Profit'])

        # State variables
        state = [
            wholesale_price,
            buyback_price,
            revenue_share,
            previous_retailer_profit
        ]

        # Actions
        if contract_type == "wholesale":
            manufacturer_action = wholesale_price
        else:
            # For 'buyback' and 'revenue-sharing' contracts
            if contract_type == "buyback":
                manufacturer_action = (wholesale_price, buyback_price)
            elif contract_type == "revenue-sharing":
                manufacturer_action = (wholesale_price, revenue_share)
            else:
                manufacturer_action = (wholesale_price, 0.0)  # Default

        retailer_action = stock

        # Rewards
        manufacturer_reward = realized_mfg_profit
        retailer_reward = realized_retailer_profit

        # Next state (use next period's data if available, else same state)
        next_state = state.copy()

        data.append((state, manufacturer_action, retailer_action, (manufacturer_reward, retailer_reward), next_state))
    return data

print("Wholesale Data Columns:", historical_data_wholesale.columns)
print("Buyback Data Columns:", historical_data_buyback.columns)
print("Revenue Sharing Data Columns:", historical_data_revenue_sharing.columns)

# Load historical data
historical_data_wholesale = pd.read_csv('../Notebooks/experiment/curr_data.csv')
historical_data_buyback = pd.read_csv('../Notebooks/experiment/curr_data_bb.csv')
historical_data_revenue_sharing = pd.read_csv('../Notebooks/experiment/curr_data_rs.csv')

# Clean column names
historical_data_wholesale.columns = historical_data_wholesale.columns.str.strip().str.replace(' ', '_')
historical_data_buyback.columns = historical_data_buyback.columns.str.strip().str.replace(' ', '_')
historical_data_revenue_sharing.columns = historical_data_revenue_sharing.columns.str.strip().str.replace(' ', '_')

# Process historical data
historical_data_wholesale = preprocess_historical_data(historical_data_wholesale, "wholesale")
historical_data_buyback = preprocess_historical_data(historical_data_buyback, "buyback")
historical_data_revenue_sharing = preprocess_historical_data(historical_data_revenue_sharing, "revenue-sharing")

# Combine historical data
historical_data = historical_data_wholesale + historical_data_buyback + historical_data_revenue_sharing

# Initialize the environment
env = SupplyChainEnv(contract_type="wholesale")  # You can change the contract type as needed

# Initialize agents with personality traits
manufacturer_agent = QLearningAgent(env.manufacturer_action_space, personality_traits=sample_manufacturer_traits)
retailer_agent = QLearningAgent(env.retailer_action_space, personality_traits=sample_retailer_traits)

# Pre-train the agents using historical data
for record in historical_data:
    state, manufacturer_action, retailer_action, rewards, next_state = record
    manufacturer_reward, retailer_reward = rewards

    # Update Q-table for manufacturer
    state_key = manufacturer_agent._state_to_key(state)
    manufacturer_agent.q_table[(state_key, manufacturer_action)] = manufacturer_reward

    # Update Q-table for retailer
    state_key = retailer_agent._state_to_key(state)
    retailer_agent.q_table[(state_key, retailer_action)] = retailer_reward

# Main training loop
num_episodes = 100  # Number of episodes for training
for episode in range(num_episodes):
    state = env.reset()
    done = False

    while not done:
        # Agents select actions
        manufacturer_action = manufacturer_agent.get_action(state)
        retailer_action = retailer_agent.get_action(state)

        # Environment step
        next_state, rewards, done, info = env.step((manufacturer_action, retailer_action))
        manufacturer_reward, retailer_reward = rewards

        # Update estimated human traits (assuming retailer is human for this example)
        # In this example, both agents are AI, so we can simulate human actions
        # For real human interaction, you would get human_action from user input
        manufacturer_agent.update_estimated_human_traits(retailer_action)
        retailer_agent.update_estimated_human_traits(manufacturer_action)

        # Update Q-tables
        manufacturer_agent.update_q_table(state, manufacturer_action, manufacturer_reward, next_state)
        retailer_agent.update_q_table(state, retailer_action, retailer_reward, next_state)

        # Move to the next state
        state = next_state

    # Optionally, print progress
    print(f"Episode {episode + 1}/{num_episodes} completed.")

# The agents are now trained and can be used to play the game against a human
# For example, to have the AI manufacturer play against a human retailer:

# Initialize environment
env = SupplyChainEnv(contract_type="wholesale")  # Adjust contract type as needed

# Reset environment
state = env.reset()
done = False

print("Game start. You are the Retailer.")

while not done:
    # Manufacturer (AI) selects action
    manufacturer_action = manufacturer_agent.get_action(state)

    # Human (Retailer) input
    print(f"Manufacturer offers wholesale price: {manufacturer_action}")
    retailer_action = int(input("Enter your order quantity (0-150): "))

    # Ensure the action is within the valid range
    retailer_action = max(0, min(env.retailer_action_space.n - 1, retailer_action))

    # Environment step
    next_state, rewards, done, info = env.step((manufacturer_action, retailer_action))
    manufacturer_reward, retailer_reward = rewards

    # Update estimated human traits
    manufacturer_agent.update_estimated_human_traits(retailer_action)

    # Update Q-table for manufacturer agent
    manufacturer_agent.update_q_table(state, manufacturer_action, manufacturer_reward, next_state)

    # Display results
    print(f"Demand was: {env.demand}")
    print(f"Your profit: {retailer_reward}, Manufacturer profit: {manufacturer_reward}")

    # Move to the next state
    state = next_state

print("Game over.")
